In [111]:
from pyspark import SparkContext
from math import sqrt
from pprint import pprint


In [112]:
sc = SparkContext('local[*]','My App Name')
data = [0, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
rDD = sc.parallelize(data, 4)


In [82]:
rDD.map(lambda x: round(sqrt(x),1)).filter(lambda x: x > 1).collect()

[1.4, 2.0, 2.8, 4.0, 5.7, 8.0, 11.3, 16.0, 22.6, 32.0]

In [84]:
rDD.map(lambda x: 1/x if x !=0 else 1E7).reduce(lambda x, y: x+y)

10000001.999023438

In [115]:
rDD2 = sc.textFile('data/part-*.txt', 10)
broken_lines = sc.accumulator(0)

def get_status(line):
    global broken_lines
    try:
        status = int(line.split()[-2])
    except:
        status = -1
        broken_lines += 1
    return status

pprint(rDD2.map(get_status).filter(lambda x: x != -1).distinct().collect())
print(broken_lines)

[404, 302, 200, 304]
3


In [116]:
rDD3 = rDD2.map(get_status)\
           .map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y)
rDD3.cache()
rDD3.take(10)

[(404, 2), (302, 2), (200, 115), (304, 11), (-1, 3)]

In [119]:
rDD3.filter(lambda x: x[0] != -1).takeOrdered(4, key=lambda x: -x[1])

[(200, 115), (304, 11), (404, 2), (302, 2)]